#### Basic Phantom GIRF measurement
Make a simple GIRF sequence that measures a series of slightly different chirp waveforms.  5 Slices are used and 4 averages, and it takes about 18 minutes to acquire.  

In [1]:
import pickle
from pathlib import Path

import numpy as np
from helpers import (
    PSeq_Base,
    PSeq_Excite_PE,
    PSeq_Moments3D,
    PSeq_Params,
    PSeq_TestWave_Chirp,
)

In [2]:
# Setup base sequernce parameters
params = PSeq_Params(channels=('x', 'y', 'z'), max_grad=32, max_slew=140)

# This holds the full sequence
pseq0 = PSeq_Base(params)

# Slice excitation
pseq_excite = PSeq_Excite_PE(params)

# Setup chirp test waveforms
pseq_test = PSeq_TestWave_Chirp(params)

# Refocus the test waveforms and any other spatial encoding
pseq_refocus = PSeq_Moments3D(
    params,
    start_areas=(None, None, pseq_test.all_areas),
    end_areas=(0, 0, 0),
    start_polarities=(1, 1, (-1, 0, 1)),
)

# Other parameters:
N_av = 4
TR = 500e-3

# Slice offsets
FOVz = 80e-3
Nslices = 5
slice_shift = 1e-3
all_offsets = np.linspace(-FOVz/2, FOVz/2, Nslices) + slice_shift


In [3]:
all_meas = []

pseq0.reinit_seq()
for i_av in range(N_av):
    for channels in [("y", "z", "x"), ("x", "z", "y"), ("x", "y", "z")]:
        params.channels = channels
        for wave_idx in range(pseq_test.N_waves):
            # 0 polarity can be removed to save time, but it may be useful in some types of processing
            for polarity in [-1, 0, 1]:  
                for offset in all_offsets:
                
                    pseq0.channels = channels

                    pseq0.track_time = 0
                    pseq0 += pseq_excite.build_blocks(offset=offset)
                    pseq0 += pseq_test.build_blocks(idx=wave_idx, polarity=polarity)
                    pseq0 += pseq_refocus.build_blocks(idx2=wave_idx, start_pol2=polarity)

                    pseq0.add_delay(TR-pseq0.track_time)
                    params.increment_rf_spoiling()
                    
                    all_meas.append(
                        {'channels': channels,
                        'wave_idx': wave_idx,
                        'polarity': polarity,
                        'i_av': i_av,
                        },
                    )

In [4]:
seq_name = f'phantom_girf_v1_{N_av}av'
print(f'======== {seq_name} ========')

pseq0.seq.set_definition(key="Name", value=seq_name)
# FOV will show up on the scanner as the slice extents.  So in plane FOV will be hidden
# (If you are using spatial encoding, if not there is no in-plane FOV)
pseq0.seq.set_definition(key="FOV", value=[FOVz, FOVz, FOVz])  

pseq0.check()

save_res = {'all_meas': all_meas}
Path('./seqs/').mkdir(parents=True, exist_ok=True)
pseq0.seq.write(f'./seqs/{seq_name}.seq')
with Path(f'./seqs/{seq_name}.pickle').open('wb') as handle:
    pickle.dump(save_res, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(' * SAVED! *')

======== phantom_girf_v1_4av ========
Seq Time: 0:18:00
Timing check passed successfully
 * SAVED! *
